# Introduction to Patent_Client

The patent_client package provides easy access to a number of USPTO and EPO patent data resources, all through a familiar ORM-like interface. If you have experience with the [Django ORM](https://docs.djangoproject.com/en/2.1/topics/db/queries/#retrieving-objects) with some extra features to accomodate the various data sources. Each data source is encapsulated in a Model object that represents various API's. The top-level models are:

- `USApplication` (USPTO Patent Examination Data System)
- `PtabTrial` (USPTO PTAB Trials API)
- `Assignment` (USPTO Assignments API)
- `Inpadoc` (EPO Inpadoc API)

Several of these models also have sub-models, as discussed below. This notebook shows some simple examples for doing data lookups with each of the models. The other notebooks in the repository show more complex analysis using the [Pandas](https://pandas.pydata.org/) data science library.

## USApplication (USPTO Patent Examination Data System)

The first model we'll look at is USApplication, an abstraction on top of the [USPTO Patent Examination Data System API](https://ped.uspto.gov/peds/#/apiDocumentation). The first step, which is common to all models, is to import the model object - USApplication. 

In [1]:
from patent_client import USApplication

The model can then be queried to get actual data back to do analysis, review, etc. All queries take one of two forms:

- Model.objects.filter(query) -> retreives many objects
- Model.objects.get(query) -> retreives one object, otherwise throws an error

For example, we can query USApplications by applicant by passing the query:
```python
    USApplication.objects.filter(first_named_applicant='Tesla Motors')
```

This query will retrieve all applications that name Tesla Motors as an applicant. By default, if you pass in only an argument without a keyword, like this:

```python
    USApplication.objects.get("10123456")
```
the model assumes that you're referring to the application number, and fetches the corresponding app. A complete list of available query fields is available at Model.objects.query_fields. For example:

```python
    USApplication.objects.query_fields
```

*NOTE:* If you ever need help with something, try calling the Python builtin "help" function. All models have (or should have) super-handy help text about queries, available information, related objects, etc. E.g.:

```python
    help(USApplication)
```  

To protect the innocent, we'll use Tesla's patent portfolio as an example - a company that has publicly agreed not to assert its patents against others. To do a lookup by applicant, we query USApplication object like this:

In [2]:
tesla_apps = USApplication.objects.filter(first_named_applicant='Tesla Motors')
print('We found apps with these applicant names:')
print(list(set(a.applicants[0].name for a in tesla_apps)))

We found apps with these applicant names:
['Inc.; Tesla Motors', 'TESLA MOTORS. INC.;', 'Tesla Motors, Inc.;', 'TESLA MOTORS CANADA ULC;', 'Tesla Motors Canada ULC;', 'TESLA MOTORS, INC.;', 'TESLA MOTORS, INC;']


Let's grab one case, and explore the data therein. Note that all Model objects are dataclasses, with all the magic that includes, including descriptive __str__ functions with basic data:

In [3]:
tesla_app = tesla_apps[15]
tesla_app

USApplication(appl_id='16057119', patent_title='NOVEL BATTERY SYSTEMS BASED ON TWO-ADDITIVE ELECTROLYTE SYSTEMS INCLUDING 1,2,6-OXODITHIANE-2,2,6,6-TETRAOXIDE', app_status='Non Final Action Mailed')

A complete list of data attributes can be found by examining the fields of the dataclass

In [4]:
[f.name for f in tesla_app.fields()]

['appl_id',
 'app_filing_date',
 'patent_title',
 'app_status',
 'app_status_date',
 'app_early_pub_number',
 'app_early_pub_date',
 'patent_number',
 'patent_issue_date',
 'wipo_early_pub_number',
 'wipo_early_pub_date',
 'corr_addr_cust_no',
 'app_cust_number',
 'app_attr_dock_number',
 'app_location',
 'first_inventor_file',
 'app_type',
 'app_entity_status',
 'app_confr_number',
 'app_cls_sub_cls',
 'app_grp_art_number',
 'app_exam_name',
 'pta_pte_summary',
 'correspondent',
 'inventors',
 'applicants',
 'attorneys',
 'transactions',
 'child_continuity',
 'parent_continuity',
 'foreign_priority',
 'pta_pte_tran_history',
 'assignments']

As you can see, there is a wealth of data about this particular case. Most of the names are relatively self-explanatory. We can look at some basic data by grabbing all the elements that start with "app". First, we'll use the .to_dict() method to convert the application to a standard Python dictionary. Then we'll only print items if they contain "_app":

In [5]:
for k, v in tesla_app.to_dict().items():
    if 'app_' in k or k == 'appl_id':
        print(f'{k}: {v}')
        

app_attr_dock_number: TSLA.005A2
app_cls_sub_cls: 429/324000
app_confr_number: 2162
app_cust_number: 113241
app_early_pub_date: 2019-09-12
app_early_pub_number: US20190280334A1
app_entity_status: UNDISCOUNTED
app_exam_name: YANCHUK, STEPHEN J
app_filing_date: 2018-08-07
app_grp_art_number: 1723
app_location: ELECTRONIC
app_status: Non Final Action Mailed
app_status_date: 2022-07-19
app_type: Utility
appl_id: 16057119


There are also some composite data elements. These elements consist of several related Model objects that model some part of the Application data. These data elements are:
    
- `USApplication.transactions` (a list of Transaction objects)
- `USApplication.children` and `USApplication.parents` (both are lists of Relationship objects)
- `USApplication.foreign_priority` (a list of ForeignPriority objects)
- `USApplication.pta_pte_history` (a list of PtaPteHistory objects)
- `USApplication.pta_pte_summary` (a PtaPteSummary object)
- `USApplication.correspondent` (a Correspondent object)
- `USApplication.attorneys` (a list of Attorney objects)
    
To use as an example, let's take a look at the transaction data, and fetch the first 15 records. 

In [6]:
tesla_app.transactions[:15]

[Transaction(date=datetime.date(2022, 7, 21), code='ELC_RVW', description='Electronic Review'),
 Transaction(date=datetime.date(2022, 7, 21), code='EML_NTF', description='Email Notification'),
 Transaction(date=datetime.date(2022, 7, 21), code='MCTNF', description='Mail Non-Final Rejection'),
 Transaction(date=datetime.date(2022, 7, 15), code='CTNF', description='Non-Final Rejection'),
 Transaction(date=datetime.date(2022, 7, 15), code='IDSC', description='Information Disclosure Statement considered'),
 Transaction(date=datetime.date(2022, 7, 15), code='IDSC', description='Information Disclosure Statement considered'),
 Transaction(date=datetime.date(2021, 12, 20), code='WIDS', description='Information Disclosure Statement (IDS) Filed'),
 Transaction(date=datetime.date(2021, 7, 23), code='M844', description='Information Disclosure Statement (IDS) Filed'),
 Transaction(date=datetime.date(2021, 7, 26), code='FWDX', description='Date Forwarded to Examiner'),
 Transaction(date=datetime.dat

You might notice that the above data is relatively messy. We can make the data look a bit cleaner by converting the objects to a Pandas dataframe. Every list of objects in patent_client has a .to_pandas method that will convert it to a pandas dataframe:

In [7]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # makes sure the table runs all the way to the edge

tesla_app.transactions[:15].to_pandas()

,code,date,description
0,ELC_RVW,2022-07-21,Electronic Review
1,EML_NTF,2022-07-21,Email Notification
2,MCTNF,2022-07-21,Mail Non-Final Rejection
3,CTNF,2022-07-15,Non-Final Rejection
4,IDSC,2022-07-15,Information Disclosure Statement considered
5,IDSC,2022-07-15,Information Disclosure Statement considered
6,WIDS,2021-12-20,Information Disclosure Statement (IDS) Filed
7,M844,2021-07-23,Information Disclosure Statement (IDS) Filed
8,FWDX,2021-07-26,Date Forwarded to Examiner
9,RCEX,2021-07-23,Request for Continued Examination (RCE)


## PtabProceeding (USPTO PTAB Api)

Now that you've seen a little bit of the USApplication API, let's take a look at the PTAB Trials API. We'll start with the usual import:

In [8]:
from patent_client import PtabProceeding

For randomness' sake, we'll pick the first IPR of 2017 to play around with, and look it up by its trial number:

In [9]:
trial = PtabProceeding.objects.get('IPR2017-00001')
print(f'PTAB Trial No. {trial.proceeding_number} was filed {trial.accorded_filing_date} to challenge U.S. Patent {trial.respondent_patent_number}')
print(f'The case is styled {trial.petitioner_party_name} v. {trial.respondent_party_name}')

PTAB Trial No. IPR2017-00001 was filed 2016-10-01 to challenge U.S. Patent 7468661
The case is styled Emerson Electric Co. v. SIPCO, LLC


The complete list of available data is again available at .attrs:

In [10]:
[f.name for f in trial.fields()]

['last_modified_date',
 'last_modified_user_id',
 'institution_decision_date',
 'proceeding_filing_date',
 'accorded_filing_date',
 'proceeding_status_category',
 'proceeding_number',
 'proceeding_last_modified_date',
 'proceeding_type_category',
 'subproceeding_type_category',
 'decision_date',
 'docket_notice_mail_date',
 'declaration_date',
 'style_name_text',
 'respondent_technology_center_number',
 'respondent_patent_owner_name',
 'respondent_party_name',
 'respondent_group_art_unit_number',
 'respondent_inventor_name',
 'respondent_counsel_name',
 'respondent_grant_date',
 'respondent_patent_number',
 'respondent_application_number_text',
 'respondent_publication_number',
 'respondent_publication_date',
 'petitioner_technology_center_number',
 'petitioner_patent_owner_name',
 'petitioner_party_name',
 'petitioner_group_art_unit_number',
 'petitioner_inventor_name',
 'petitioner_counsel_name',
 'petitioner_grant_date',
 'petitioner_patent_number',
 'petitioner_application_number_t

Where possible, there are also related objects that can be fetch from a Model object. For example, the challenged patent has a related USApplication. A complete list of all related objects and other data can be found by calling the help function on the object:

In [11]:
help(trial)

Help on PtabProceeding in module patent_client.uspto.ptab.model object:

class PtabProceeding(patent_client.util.model.Model)
 |  PtabProceeding(last_modified_date: 'datetime.datetime' = None, last_modified_user_id: 'datetime.datetime' = None, institution_decision_date: 'datetime.date' = None, proceeding_filing_date: 'datetime.date' = None, accorded_filing_date: 'datetime.date' = None, proceeding_status_category: str = None, proceeding_number: str = None, proceeding_last_modified_date: 'datetime.date' = None, proceeding_type_category: str = None, subproceeding_type_category: str = None, decision_date: 'datetime.date' = None, docket_notice_mail_date: 'datetime.date' = None, declaration_date: 'datetime.date' = None, style_name_text: str = None, respondent_technology_center_number: str = None, respondent_patent_owner_name: str = None, respondent_party_name: str = None, respondent_group_art_unit_number: str = None, respondent_inventor_name: str = None, respondent_counsel_name: str = None, 

From the data above, we see that there is a related data object called "us_application". That is, we can fetch the application object that corresponds to the PtabTrial simply by retreiving the attribute .us_application, like so:

In [12]:
trial_app = trial.us_application
print(trial_app)
print({k:v for (k, v) in trial_app.to_dict().items() if 'app_' in k})

USApplication(appl_id='11395685', patent_title='SYSTEM AND METHOD FOR MONITORING AND CONTROLLING REMOTE DEVICES', app_status='Patented Case')
{'app_attr_dock_number': 'STAT3', 'app_cls_sub_cls': '340/531000', 'app_confr_number': '1922', 'app_cust_number': '89818', 'app_early_pub_date': datetime.date(2006, 8, 17), 'app_early_pub_number': 'US20060181406A1', 'app_entity_status': 'UNDISCOUNTED', 'app_exam_name': 'PHAM, TOAN NGOC', 'app_filing_date': datetime.date(2006, 3, 31), 'app_grp_art_number': '2612', 'app_location': 'ELECTRONIC', 'app_status': 'Patented Case', 'app_status_date': datetime.date(2008, 12, 3), 'app_type': 'Utility'}
